In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from datetime import date
import re
from dateutil.relativedelta import relativedelta

In [27]:
aapl = yf.Ticker("aapl")
aapl.option_chain(date="2022-08-19").calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL220819C00030000,2022-08-11 19:14:15+00:00,30.0,138.50,141.60,141.85,0.0,0.0,47,186,0.000010,True,REGULAR,USD
1,AAPL220819C00035000,2022-08-11 16:45:53+00:00,35.0,134.40,136.65,136.85,0.0,0.0,4,58,0.000010,True,REGULAR,USD
2,AAPL220819C00040000,2022-08-10 13:43:46+00:00,40.0,127.75,131.65,131.95,0.0,0.0,1,1,0.000010,True,REGULAR,USD
3,AAPL220819C00045000,2022-08-04 19:58:31+00:00,45.0,120.70,126.65,127.00,0.0,0.0,105,0,0.000010,True,REGULAR,USD
4,AAPL220819C00050000,2022-08-11 17:20:45+00:00,50.0,119.55,121.75,122.05,0.0,0.0,6,53,0.000010,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,AAPL220819C00240000,2022-08-09 16:28:09+00:00,240.0,0.01,0.00,0.01,0.0,0.0,89,1706,0.718753,False,REGULAR,USD
64,AAPL220819C00245000,2022-06-10 17:36:21+00:00,245.0,0.02,0.00,0.03,0.0,0.0,373,1369,0.828127,False,REGULAR,USD
65,AAPL220819C00250000,2022-08-03 17:05:38+00:00,250.0,0.01,0.00,0.01,0.0,0.0,10,893,0.796877,False,REGULAR,USD
66,AAPL220819C00255000,2022-06-22 14:11:39+00:00,255.0,0.03,0.00,0.01,0.0,0.0,1,860,0.843752,False,REGULAR,USD


In [9]:
stock_data = yf.download("aapl", (date.today() - relativedelta(days=30)).strftime("%Y-%m-%d"), date.today().strftime("%Y-%m-%d"), adjusted=True)
stock_returns = stock_data['Adj Close'].pct_change().dropna()
# stock_returns

[*********************100%***********************]  1 of 1 completed


In [16]:
def symbol_parts(symbol):
    r = re.search("^[A-Z]*", symbol)
    index = r.span()[1]
    ticker = symbol[0:index]
    year = 2000+ int(symbol[index:index + 2])
    month = int(symbol[index+2: index+4])
    day = int(symbol[index+4: index+6])
    opt_type = symbol[index+6: index+7]
    strike_price = int(symbol[index+7:]) / 1000
    return (ticker,date(year, month, day),opt_type,strike_price)


In [17]:
def get_sigma(stock_data):
    log_returns = np.array([stock_data[i+1]/stock_data[i] for i in range(len(stock_data)-1)])
    return np.sqrt(np.mean(log_returns**2))

def get_d1(sigma, t, s, x, r):
    return 1/(sigma * np.sqrt(t)) * (np.log(s/x) + (r + sigma**2/2)*t)

In [41]:
def option_price(symbol, r):
    ticker,t,option_type, x = symbol_parts(symbol)
    t = (t-date.today()).days/365
    stock_data = yf.download(ticker, (date.today() - relativedelta(days=30)).strftime("%Y-%m-%d"), date.today().strftime("%Y-%m-%d"), adjusted=True)['Adj Close']
    s = stock_data[-1]
    sigma = get_sigma(stock_data)
    d1 = get_d1(sigma, t, s,x,r)
    d2 = d1 - sigma * np.sqrt(t)
    return norm.cdf(d1)*s - norm.cdf(d2)*x*np.exp(-1*r*t)

option_price("AAPL220819C00030000", .091)

[*********************100%***********************]  1 of 1 completed


138.54231599798078

In [44]:
aapl = yf.Ticker("aapl")
calls = aapl.option_chain(date="2022-08-19").calls

priced_options = pd.DataFrame(columns=["contractSymbol", "price", "ask"])
for i in range(calls.shape[0]):
    price = option_price(calls.iloc[i].contractSymbol, .091)
    if price > calls.iloc[i].ask:
        priced_options.loc[i] = [calls.iloc[i].contractSymbol, price, calls.iloc[i].ask]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [54]:
pd.to_pickle(priced_options, "priced_options.pickle")